In [1]:
#######################################################
##### STRIP TWEET #####################################
#######################################################

#import pandas as pd
#df = pd.read_csv("train2017.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
#corpus = df[3].tolist()

import nltk
import string
import re

f = open("small.tsv")
line = f.readline()

tweets = []
translate_table = dict((ord(char), None) for char in string.punctuation)   
cnt = 1
while line:
    pretext = ' '.join(line.split()[3:])
    pretext = ' '.join(word for word in pretext.split(' ') if not word.startswith('@')) #remove @user
    pretext = re.sub(r"http\S+", "", pretext) #remove link
    pretext = pretext.translate(translate_table) #remove symbols 
    #remove numbers??
    
    tweet = pretext.lower()
    tweets.append(tweet)
    line = f.readline()
    cnt += 1

#print(tweets)

In [2]:
#######################################################
##### TOKENIZATION ####################################
#######################################################

from nltk import word_tokenize

tokens = []
for tweet in tweets:
    token = []
    token = word_tokenize(tweet)
    tokens.append(token)

print(tokens)

[['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'going', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], ['theo', 'walcott', 'is', 'still', 'shit', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], ['its', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], ['iranian', 'general', 'says', 'israels', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missiles', 'keep', 'talking', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'finding', 'out'], ['tehran', 'mon', 'amour', 'obama', 'tried', 'to', 'establish', 'ties', 'with', 'the', 'mullahs', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], ['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], ['mashed', 'out', 'to', 'niggas', 'in', 'paris', 'in', 'the', 'club', 'while', 'in', 'paris', 'as', 'cliche', 'as', 'it', 'may', 'sound', 'weouthere'], ['larry', 'bird', 'is', 'ranked', 

In [3]:
from nltk import pos_tag

from nltk.corpus import wordnet
from nltk.stem import  WordNetLemmatizer
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

tweets = []
for token_list in tokens:
    lemmatized = []
    for word in token_list:
        lemmatized.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
    tweets.append(lemmatized)

#Rprint(tweets)

In [4]:
corpus = []
for tweet in tweets:
    corpus.append(" ".join(str(word) for word in tweet))

print(corpus)

['gas by my house hit 339 im go to chapel hill on sat 😂', 'theo walcott be still shit watch rafa and johnny deal with him on saturday', 'it not that im a gsp fan i just hate nick diaz cant wait for february', 'iranian general say israel iron dome cant deal with their missile keep talk like that and we may end up find out', 'tehran mon amour obama try to establish tie with the mullah via no barack obama vote mitt romney', 'i sat through this whole movie just for harry and ron at christmas ohlawd 911', 'mash out to nigga in paris in the club while in paris a cliche a it may sound weouthere', 'larry bird be ranked 4th alltime not include lebron or kobe just sayin']


In [5]:
#BAG-OF-WORDS VECTORIZATION
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_BOW = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
#print(X.toarray()[0])
#print(X_BOW.toarray())


In [16]:
#TF-IDF VECTORIZATION
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_TFIDF = vectorizer.fit_transform(corpus)
#print(X_TFIDF.toarray())


AttributeError: 'list' object has no attribute 'split'

In [36]:
#we have 3 options: word2vec, word2vec in sklearn, doc2vec
from gensim.test.utils import common_texts

#IMPORT WORD2VEC
from gensim.models import Word2Vec
nested_corpus = [corpus[i:i+1] for i in range(0, len(corpus), 1)]
model = Word2Vec(tokens, size=100, window=5, min_count=1, workers=4) #size of vector is 100
model.train(tokens, total_examples=model.corpus_count,epochs=model.epochs)  # train word vectors
print(model['gas'])

#IMPORT WORD2VEC LIBRARY AVAILABLE IN SKLEARN

#from gensim.sklearn_api import W2VTransformer
#total = sum(tokens,[])
#vectorizer = W2VTransformer(size=10, min_count=1, seed=1)
#print(corpus)
#X_embeddings = vectorizer.fit(total).transform(tokens)



[['gas by my house hit 339 im go to chapel hill on sat 😂'], ['theo walcott be still shit watch rafa and johnny deal with him on saturday'], ['it not that im a gsp fan i just hate nick diaz cant wait for february'], ['iranian general say israel iron dome cant deal with their missile keep talk like that and we may end up find out'], ['tehran mon amour obama try to establish tie with the mullah via no barack obama vote mitt romney'], ['i sat through this whole movie just for harry and ron at christmas ohlawd 911'], ['mash out to nigga in paris in the club while in paris a cliche a it may sound weouthere'], ['larry bird be ranked 4th alltime not include lebron or kobe just sayin']]
[ 0.00118727 -0.00420469  0.00284674  0.00052224  0.00062534 -0.00345467
 -0.00357037 -0.00089545  0.00340331 -0.00465742  0.00050744 -0.00355314
  0.00107752  0.004303   -0.0027083  -0.0046858  -0.00407666  0.0037294
 -0.00420923  0.00044863 -0.0049019  -0.0017915  -0.0006112   0.00346838
  0.00419763 -0.002361

/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [55]:
#Doc2vec is equal with word2vec but is more appropriate for phrases (vectorize phrases instead of words). I.e 
#1.Manos leaves the office every day at 18:00 to catch his train
#2. This season is called Fall, because leaves fall from the trees.
#In this way we can capture the difference between the same word used in a different context. For example we now have a
#different representation of the word “leaves” in the above two sentences


#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(corpus)]
print(tagged_data)
model = Doc2Vec(size=200,
                alpha=model.alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

max_epoch = 20
for epoch in range(max_epoch):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
X_embeddings = model.docvecs   
print(X_embeddings['1']) 

[TaggedDocument(words=['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'go', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], tags=['0']), TaggedDocument(words=['theo', 'walcott', 'be', 'still', 'shit', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], tags=['1']), TaggedDocument(words=['it', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], tags=['2']), TaggedDocument(words=['iranian', 'general', 'say', 'israel', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missile', 'keep', 'talk', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'find', 'out'], tags=['3']), TaggedDocument(words=['tehran', 'mon', 'amour', 'obama', 'try', 'to', 'establish', 'tie', 'with', 'the', 'mullah', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], tags=['4']), TaggedDocument(words=['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], tags=['5']),

/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


[ 8.58764257e-03 -8.66686460e-04 -2.43878472e-04 -2.77113332e-03
  9.10880137e-03 -1.15173019e-03  3.66792339e-03 -1.97636969e-02
 -1.05814030e-02  1.84497680e-03 -7.06963753e-03  1.30652627e-02
  6.02251012e-03  3.64462286e-03  1.05028029e-03  1.04968362e-02
 -4.17568581e-03 -5.79190906e-03 -2.13834178e-02 -1.37677118e-02
  4.71049396e-04  6.02357881e-03  2.83132773e-03 -3.24094319e-03
  2.04147218e-04  5.33911772e-03 -5.49137732e-03 -2.88123498e-03
  2.19008215e-02 -4.01217071e-03 -1.14812301e-02 -4.38141637e-03
 -5.50785661e-03 -2.14982964e-03 -2.19534617e-03 -5.77179994e-03
  5.75564161e-04 -3.34441173e-03 -2.15831175e-02  1.79384917e-03
  3.02760280e-03 -8.33137706e-03 -3.12308175e-03  1.19119911e-02
  5.20762196e-03  3.52577073e-03 -9.33160016e-04  2.04997906e-03
 -1.62148126e-03 -7.01380195e-03  6.67410018e-03  8.59002525e-04
 -1.20499087e-02 -7.86115509e-03 -2.23006345e-02  6.66019879e-03
 -2.66672811e-03  6.57899678e-03 -1.42436894e-02 -1.64177995e-02
  5.38725266e-03  5.35216